# Reinforcement Learning - Monte Carlo
If you want to test/submit your solution **restart the kernel, run all cells and submit the mc_autograde.py file into codegrade.**

In [1]:
# This cell imports %%execwritefile command (executes cell and writes it into file). 
# All cells that start with %%execwritefile should be in mc_autograde.py file after running all cells.
from custommagics import CustomMagics
get_ipython().register_magics(CustomMagics)

In [2]:
%%execwritefile mc_autograde.py
import numpy as np
from collections import defaultdict
from tqdm import tqdm as _tqdm

def tqdm(*args, **kwargs):
    return _tqdm(*args, **kwargs, mininterval=1)  # Safety, do not overflow buffer

Overwriting mc_autograde.py


In [3]:
import matplotlib.pyplot as plt
import sys


%matplotlib inline

assert sys.version_info[:3] >= (3, 6, 0), "Make sure you have Python 3.6 installed!"

## 1. Monte Carlo Prediction

For the Monte Carlo Prediction we will look at the Blackjack game (Example 5.1 from the book), for which the `BlackjackEnv` is implemented in `blackjack.py`. Note that compared to the gridworld, the state is no longer a single integer, which is why we use a dictionary to represent the value function instead of a numpy array. By using `defaultdict`, each state gets a default value of 0.

In [4]:
from blackjack import BlackjackEnv
env = BlackjackEnv()

For the Monte Carlo algorithm, we no longer have transition probabilities and we need to *interact* with the environment. This means that we start an episode by using `env.reset` and send the environment actions via `env.step` to observe the reward and next observation (state).

In [5]:
# So let's have a look at what we can do in general with an environment...
import gym
?gym.Env

In [6]:
# We can also look at the documentation/implementation of a method
?env.step

In [7]:
??BlackjackEnv

A very simple policy for Blackjack is to *stick* if we have 20 or 21 points and *hit* otherwise. We want to know how good this policy is. This policy is *deterministic* and therefore a function that maps an observation to a single action. Technically, we can implement this as a dictionary , a function or a class with a function, where we use the last option. Moreover, it is often useful (as you will see later) to implement a function that returns  the probability $\pi(a|s)$ for the state action pair (the probability that this policy would perform certain action in given state). We group these two functions in a policy class. To get started, let's implement this simple policy for BlackJack.

In [8]:
%%execwritefile -a mc_autograde.py

class SimpleBlackjackPolicy(object):
    """
    A simple BlackJack policy that sticks with 20 or 21 points and hits otherwise.
    """
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains a probability
        of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """
        probs = []
        for state, action in zip(states, actions):
            policyaction = self.sample_action(state)
            probs.append(float(policyaction == action))
        
        return np.array(probs)
    
    def sample_action(self, state):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            state: current state

        Returns:
            An action (int).
        """
        if state[0] == 20 or state[0] == 21:
            return 0
        return 1

Appending to mc_autograde.py


In [9]:
# Let's check if it makes sense
env = BlackjackEnv()
s = env.reset()
policy = SimpleBlackjackPolicy()
print("State: {}\nSampled Action: {}\nProbabilities [stick, hit]: {}".format(s, policy.sample_action(s), policy.get_probs([s,s],[0,1])))

State: (13, 10, False)
Sampled Action: 1
Probabilities [stick, hit]: [0. 1.]


Since there are multiple algorithms which require data from single episode (or multiple episodes) it is often useful to write a routine that will sample a single episode. This will save us some time later. Implement a *sample_episode* function which uses environment and policy to sample a single episode.

In [10]:
%%execwritefile -a mc_autograde.py

def sample_episode(env, policy):
    """
    A sampling routine. Given environment and a policy samples one episode and returns states, actions, rewards
    and dones from environment's step function and policy's sample_action function as lists.

    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.

    Returns:
        Tuple of lists (states, actions, rewards, dones). All lists should have same length. 
        Hint: Do not include the state after the termination in the list of states.
    """
    states = []
    actions = []
    rewards = []
    dones = []
    
    states.append(env.reset())
    t = 0
    while True:
        a = policy.sample_action(states[t])
        sp, r, done, _ = env.step(a)
        
        actions.append(a)
        rewards.append(r)
        dones.append(done)
        if done:
            break
        
        states.append(sp)
        t += 1
    
    return states, actions, rewards, dones

Appending to mc_autograde.py


In [11]:
# Let's sample some episodes
env = BlackjackEnv()
policy = SimpleBlackjackPolicy()
for episode in range(3):
    trajectory_data = sample_episode(env, policy)
    print("Episode {}:\nStates {}\nActions {}\nRewards {}\nDones {}\n".format(episode,*trajectory_data))

Episode 0:
States [(18, 10, False), (19, 10, False), (21, 10, False)]
Actions [1, 1, 0]
Rewards [0, 0, 1]
Dones [False, False, True]

Episode 1:
States [(21, 5, True)]
Actions [0]
Rewards [1]
Dones [True]

Episode 2:
States [(19, 10, False)]
Actions [1]
Rewards [-1]
Dones [True]



Now implement the MC prediction algorithm (either first visit or every visit). Hint: you can use `for i in tqdm(range(num_episodes))` to show a progress bar. Use the sampling function from above to sample data from a single episode.

In [12]:
%%execwritefile -a mc_autograde.py

def mc_prediction(env, policy, num_episodes, discount_factor=1.0, sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given policy using sampling.
    
    Args:
        env: OpenAI gym environment.
        policy: A policy which allows us to sample actions with its sample_action method.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """

    # Keeps track of current V and count of returns for each state
    # to calculate an update.
    V = defaultdict(float)
    
    returns = defaultdict(list)
    
    returns_count = defaultdict(float)
    
    for i in tqdm(range(num_episodes)):
        S = []
        states, actions, rewards, dones = sampling_function(env, policy)
        
        G = 0
        for idx in range(len(states) - 1, -1, -1):
            G = discount_factor * G + rewards[idx]
            
            s = states[idx]
            if s not in S:
                S.append(s)
                returns[s].append(G)
                V[s] = sum(returns[s])/len(returns[s])
                
    return V

Appending to mc_autograde.py


In [13]:
V_10k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=10000)
print(V_10k)

100%|██████████| 10000/10000 [00:00<00:00, 16627.36it/s]

defaultdict(<class 'float'>, {(18, 10, False): -0.7197309417040358, (19, 10, True): -0.4032258064516129, (14, 10, True): -0.4358974358974359, (16, 9, False): -0.6055045871559633, (15, 8, False): -0.5384615384615384, (15, 10, False): -0.6303797468354431, (21, 10, False): 0.8842443729903537, (12, 8, False): -0.536, (17, 8, False): -0.6565656565656566, (20, 1, False): 0.05185185185185185, (17, 10, False): -0.7129840546697038, (21, 6, True): 0.9090909090909091, (16, 10, False): -0.6538461538461539, (13, 10, False): -0.6127320954907162, (16, 1, False): -0.723404255319149, (14, 1, False): -0.77, (15, 6, False): -0.5185185185185185, (13, 6, False): -0.5301204819277109, (17, 3, False): -0.6226415094339622, (20, 10, False): 0.391304347826087, (19, 10, False): -0.7657657657657657, (18, 8, False): -0.5934959349593496, (16, 8, False): -0.6631578947368421, (12, 2, False): -0.5904761904761905, (19, 9, False): -0.6771653543307087, (15, 9, False): -0.6153846153846154, (12, 7, False): -0.46728971962616

Now make *4 plots* like Figure 5.1 in the book. You can either make 3D plots or heatmaps. Make sure that your results look similar to the results in the book. Give your plots appropriate titles, axis labels, etc.

In [ ]:
%%time
# Let's run your code one time
V_10k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=10000)
V_500k = mc_prediction(env, SimpleBlackjackPolicy(), num_episodes=500000)

 92%|█████████▏| 459136/500000 [01:52<00:18, 2189.96it/s]

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()

noAce = []
withAce = []
for key, value in V_10k.items():
    if key[2]:
        # Take the initial player and dealer scores + calculated win chance (0 = draw, -1 = loss etc)
        withAce.append([key[0], key[1], value])
    else:
        noAce.append([key[0], key[1], value])

noAce = np.array(noAce)
withAce = np.array(withAce)


ax = fig.add_subplot(221, projection='3d')
ax.set_title("No usable ace, 10k")
ax.plot_trisurf(noAce[:, 1], noAce[:, 0], noAce[:, 2])
ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)
        
ax = fig.add_subplot(222, projection='3d')
ax.set_title("Usable ace, 10k")
ax.plot_trisurf(withAce[:, 1], withAce[:, 0], withAce[:, 2])  
ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)

noAce = []
withAce = []
for key, value in V_500k.items():
    if key[2]:
        # Take the initial player and dealer scores + calculated win chance (0 = draw, -1 = loss etc)
        withAce.append([key[0], key[1], value])
    else:
        noAce.append([key[0], key[1], value])

noAce = np.array(noAce)
withAce = np.array(withAce)


ax = fig.add_subplot(223, projection='3d')
ax.plot_trisurf(noAce[:, 1], noAce[:, 0], noAce[:, 2])
ax.set_title("No usable ace, 500k")
ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)
        
ax = fig.add_subplot(224, projection='3d')
ax.plot_trisurf(withAce[:, 1], withAce[:, 0], withAce[:, 2])    
ax.set_title("Usable ace, 500k")
ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)


plt.show()


## 2. Off-policy Monte Carlo prediction
In real world, it is often beneficial to learn from the experience of others in addition to your own. For example, you can probably infer that running off the cliff with a car is a bad idea if you consider what "return" people who have tried it received.

Similarly, we can benefit from the experience of other agents in reinforcement learning. In this exercise we will use off-policy monte carlo to estimate the value function of our target policy using the experience from a different behavior policy. Our target policy will be the simple policy defined above (stick if we have *20* or *21* points) and we will use a random policy that randomly chooses to stick or hit (both with 50% probability) as a behavior policy. As a first step, implement a random BlackJack policy.

In [ ]:
%%execwritefile -a mc_autograde.py

class RandomBlackjackPolicy(object):
    """
    A random BlackJack policy.
    """
    def get_probs(self, states, actions):
        """
        This method takes a list of states and a list of actions and returns a numpy array that contains 
        a probability of perfoming action in given state for every corresponding state action pair. 

        Args:
            states: a list of states.
            actions: a list of actions.

        Returns:
            Numpy array filled with probabilities (same length as states and actions)
        """
        
        return np.zeros(len(states)) + 0.5
    
    def sample_action(self, state):
        """
        This method takes a state as input and returns an action sampled from this policy.  

        Args:
            state: current state

        Returns:
            An action (int).
        """
        return np.random.choice(a=[0, 1])


In [ ]:
# Let's check if it makes sense
env = BlackjackEnv()
s = env.reset()
policy = RandomBlackjackPolicy()
print("State: {}\nSampled Action: {}\nProbabilities [stick, hit]: {}".format(s, policy.sample_action(s), policy.get_probs([s,s],[0,1])))

Now implement the MC prediction algorithm with ordinary importance sampling. Use the sampling function from above to sample data from a single episode.

Hint: Get probs functions may be handy. You can use `for i in tqdm(range(num_episodes))` to show a progress bar.

In [ ]:
%%execwritefile -a mc_autograde.py

def mc_importance_sampling(env, behavior_policy, target_policy, num_episodes, discount_factor=1.0,
                           sampling_function=sample_episode):
    """
    Monte Carlo prediction algorithm. Calculates the value function
    for a given target policy using behavior policy and ordinary importance sampling.
    
    Args:
        env: OpenAI gym environment.
        behavior_policy: A policy used to collect the data.
        target_policy: A policy which value function we want to estimate.
        num_episodes: Number of episodes to sample.
        discount_factor: Gamma discount factor.
        sampling_function: Function that generates data from one episode.
    
    Returns:
        A dictionary that maps from state -> value.
        The state is a tuple and the value is a float.
    """
    
    # Keeps track of current V and count of returns for each state
    # to calculate an update.
    V = defaultdict(float)
    
    returns = defaultdict(list)
    returns_count = defaultdict(float)
    
    for i in tqdm(range(num_episodes)):
        states, actions, rewards, dones = sampling_function(env, behavior_policy)
        
        pt = target_policy.get_probs(states, actions)
        pb = behavior_policy.get_probs(states, actions)
        
        
#         print(states)
#         print(actions)
#         print(pt)
#         print(pb)
        
        W = np.sum(pt/pb)
        G = 0
        
        for idx in range(len(states) - 1, -1 , -1):
            S = []
            G = discount_factor * G + rewards[idx]
            
            if pt[idx] != 0:
                W /= pt[idx]/pb[idx]
            else:
                W = 0
                break
            
            s = states[idx]
            if s not in S:
                returns_count[s] += 1
                returns[s].append(G)
                S.append(s)         
                
#                 print(W)
                
#                 print("updating V")
#                 print("W = ", W)
#                 print("Cur V =", V[s], "a = ", 1/returns_count[s], "diff = ", (W*G - V[s]))
                
                V[s] = (V[s] + (1/returns_count[s]) * (W*G - V[s]))
                
    return V

In [ ]:
V_10k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=10000)
print(max(V_10k.values()), min(V_10k.values()))

print(V_10k)

In [ ]:
%%time
# Let's run your code one time
V_10k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=10000)
V_500k = mc_importance_sampling(env, RandomBlackjackPolicy(), SimpleBlackjackPolicy(), num_episodes=500000)

Plot the V function. Do the plots look like what you expected?

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()

noAce = []
withAce = []
for key, value in V_10k.items():
    if key[2]:
        # Take the initial player and dealer scores + calculated win chance (0 = draw, -1 = loss etc)
        withAce.append([key[0], key[1], value])
    else:
        noAce.append([key[0], key[1], value])

noAce = np.array(noAce)
withAce = np.array(withAce)


ax = fig.add_subplot(221, projection='3d')
ax.set_title("No usable ace, 10k")
ax.plot_trisurf(noAce[:, 1], noAce[:, 0], noAce[:, 2])
# ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)
        
ax = fig.add_subplot(222, projection='3d')
ax.set_title("Usable ace, 10k")
ax.plot_trisurf(withAce[:, 1], withAce[:, 0], withAce[:, 2])  
# ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)

noAce = []
withAce = []
for key, value in V_500k.items():
    if key[2]:
        # Take the initial player and dealer scores + calculated win chance (0 = draw, -1 = loss etc)
        withAce.append([key[0], key[1], value])
    else:
        noAce.append([key[0], key[1], value])

noAce = np.array(noAce)
withAce = np.array(withAce)


ax = fig.add_subplot(223, projection='3d')
ax.plot_trisurf(noAce[:, 1], noAce[:, 0], noAce[:, 2])
ax.set_title("No usable ace, 500k")
# ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)
        
ax = fig.add_subplot(224, projection='3d')
ax.plot_trisurf(withAce[:, 1], withAce[:, 0], withAce[:, 2])    
ax.set_title("Usable ace, 500k")
# ax.set_zlim(-1, 1)
ax.set_ylim(10, 21)


plt.show()

If you want to test/submit your solution **restart the kernel, run all cells and submit the mc_autograde.py file into codegrade.**